## Plotting Candle Sticks

Welcome to this ipython notebook for plotting candle sticks. We are going to use numpy, quandl and plotly in order to acquire data and plot out candle sticks

In [ ]:
import talib
import numpy as np
import quandl
import plotly.plotly as py
import plotly.graph_objs as go
from datetime import datetime
import plotly
import torch
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plot

Here we can specify the specific stock info that we want. Search the quandl free api calls here(https://www.quandl.com/search?query=) Currently I have a call bellow that gets bitcoin prices on a day resolution. We get the Open, Close, High, Low, Volume(BTC), Volume(Currency) and weighted price values for each day. These are stored in a pandas data frame and can be accesed i.e. stock_val['open']. All of the data has a date value associated with it.

In [ ]:
stock_val=quandl.get("BCHARTS/BITFINEXUSD")

In [ ]:
stock_val.describe()

Next we are going to make a nice candle stick plot of the data. In order to do this you have to setup an account at plotly(https://plot.ly). After you setup an account make sure to confiure your account according to these directions(https://plot.ly/python/getting-started/). Once this is done you should be able to run the bellow and get out a simple candle stick chart. Check out other charts here(https://plot.ly/python/candlestick-charts/)

In [441]:
trace = go.Candlestick(x=stock_val.index,
                       open=stock_val.Open,
                       high=stock_val.High,
                       low=stock_val.Low,
                       close=stock_val.Close)
data = [trace]
py.iplot(data, filename='simple_candlestick')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~lthul/0 or inside your plot.ly account where it is named 'simple_candlestick'


In [442]:
RB = 100.0 * (stock_val.Close - stock_val.Open) / (stock_val.Open)
US = 100.0 * (stock_val.Close - stock_val.Open) / (stock_val.High - stock_val.Open)
LS = 100.0 * (stock_val.Close - stock_val.Open) / (stock_val.Close - stock_val.Low)
data = np.zeros((RB.size,3))
data[:,0] = RB.values
data[:,1] = US.values
data[:,2] = LS.values
inputdata = torch.Tensor(data)

In [443]:
D_in, H, D_out = 3, 2, 1


In [444]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

In [445]:
input_size, hidden_size, output_size = 10, 128, 1

In [446]:
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input.unsqueeze(0), hidden),1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return Variable(torch.zeros(1,self.hidden_size))



In [447]:
model = TwoLayerNet(D_in, H, D_out)
rnn = RNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
learning_rate = 0.0005

In [448]:
x = Variable(torch.Tensor(RB.values)).unsqueeze(0)
y = Variable(torch.Tensor(RB[10:len(RB)].values)).unsqueeze(0)


In [449]:
def train(input_tensor, output_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_tensor.size()[0]):
        output, hidden = rnn(input_tensor[i], hidden)
        loss += criterion(output, output_tensor[i])

    loss.backward()

#     for p in rnn.parameters():
#         p.data.add_(-learning_rate, p.grad.data)

    return output, loss.data[0] / input_tensor.size()[0]

import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
rnn = RNN(input_size, hidden_size, output_size)

n_iters = 1000
print_every = 10
plot_every = 100
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(0, n_iters + 1):
    output, loss = train(x[0, (iter):(iter+10)].unsqueeze(0), y[0, iter].unsqueeze(0))
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0